### 导包

In [18]:
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration
import numpy as np
from torch.utils.data import Dataset, DataLoader 
import json

### 路径及参数

In [9]:
train_path = "./DuReaderQG/train.json"
test_path = "./DuReaderQG/dev.json"
max_target_len = 32
max_source_len = 256

### 加载model和tokenizer

In [ ]:
pertrained_path = './uer/t5-base-chinese-cluecorpussmall'
tokenizer = AutoTokenizer.from_pretrained(pertrained_path)
model = T5ForConditionalGeneration.from_pretrained(pertrained_path)
tokenizer, model

d:\Programs\miniconda3\envs\llm\lib\site-packages\transformers\modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

(BertTokenizerFast(name_or_path='./uer/t5-base-chinese-cluecorpussmall', vocab_size=21229, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),
 T5ForConditionalGeneration(
   (shared): Embedding(21228, 768)
   (encoder): T5Stack(
     (embed_tokens): Embedding(21228, 768)
     (block): ModuleList(
       (0): T5Block(
         (layer): ModuleList(
           (0): T5LayerSelfAttention(
             (SelfAttention): T5Attention(
               (q): Linear(in_features=768, out_features=768, bias=False)
               (k): Linear(in_features=768, out_features=768, bias=False)
               (v): Linear(in_features=768, out_features=768, bias=False)
               (o): Linear(in_features=768, out_features=768, bias=False)
               (relative_attention_bias): Embeddi

### 自定义dataset

In [ ]:
class QADataset(Dataset):
    def __init__(self, data_path):
        self.data = []
        with open(data_path, "r") as f:
            for line in f:
                self.data.append(json.loads(line))


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        item = self.data[idx]
        input_seq = (f"问题：{item['question']}{tokenizer.sep_token}原文：{item['context']}")
        output_seq = f"答案：{item['answer']}{tokenizer.eos_token}"
        return input_seq, output_seq

In [ ]:
train_dataset = QADataset(train_path)
train_dataset[0]

Using eos_token, but it is not set yet.


('问题：仙剑奇侠传3第几集上天界[SEP]原文：第35集雪见缓缓张开眼睛，景天又惊又喜之际，长卿和紫萱的仙船驶至，见众人无恙，也十分高兴。众人登船，用尽合力把自身的真气和水分输给她。雪见终于醒过来了，但却一脸木然，全无反应。众人向常胤求助，却发现人世界竟没有雪见的身世纪录。长卿询问清微的身世，清微语带双关说一切上了天界便有答案。长卿驾驶仙船，众人决定立马动身，往天界而去。众人来到一荒山，长卿指出，魔界和天界相连。由魔界进入通过神魔之井，便可登天。众人至魔界入口，仿若一黑色的蝙蝠洞，但始终无法进入。后来花楹发现只要有翅膀便能飞入。于是景天等人打下许多乌鸦，模仿重楼的翅膀，制作数对翅膀状巨物。刚佩戴在身，便被吸入洞口。众人摔落在地，抬头发现魔界守卫。景天和众魔套交情，自称和魔尊重楼相熟，众魔不理，打了起来。',
 '答案：第35集None')

In [ ]:
def collate_fn(batch):
    batched_data = {
        "input_ids": [],
        "attention_mask": [],
        "decoder_input_ids": [],
        "labels": [],
    }
    for _, (input_seq, output_seq) in enumerate(batch):
        # tokenize输入
        inputs = tokenizer(text=input_seq, truncation=True, max_length=max_source_len, padding=True)
        # tokenize输出,并将输出的ids作为inputs的label
        output_ids = tokenizer.encode(text=output_seq, truncation=True, max_length=max_target_len)
        decoder_input_ids = output_ids[:-2] # 去掉eos和[cls]
        decoder_input_ids =  decoder_input_ids + [tokenizer.pad_token_id] * (max_target_len - len(decoder_input_ids)) # padding
        
        labels = output_ids[1: -1] # 去掉起始token和[cls]
        labels = labels + [-100] * (max_target_len - len(labels))
        
        batched_data["input_ids"].append(inputs["input_ids"])
        batched_data["attention_mask"].append(inputs["attention_mask"])
        batched_data["decoder_input_ids"].append(decoder_input_ids)
        batched_data["labels"].append(labels)
        
    for k, v in batched_data.items():
        batched_data[k] = torch.tensor(np.array(v))
    return batched_data
test_train = []
for i in range(5):
    test_train.append(train_dataset[i]) 

train_test_dataloader = DataLoader(test_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
train_test_dataloader[0]                  

Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using eos_token, but it is not set yet.


TypeError: 'DataLoader' object is not subscriptable